In [ ]:
#Associazione di un identificativo al nome dell'attività
#chiave: IdAttivita
#valore: nomeAttivita
dizionario_attivita = { 
                      '010' : 'garage',
                      '020' : 'serra',
                      '030' : 'fondazioni e solaio',
                      '040' : 'murature',
                      '050' : 'copertura',
                      '060' : 'porte e serramenti esterni',
                      '070' : 'impianto idraulico',
                      '080' : 'sanitari',
                      '090' : 'impianto elettrico',
                      '100' : 'finiture',
                      '110' : 'pitturazioni',
                      '120' : 'sistemazione della strada',
                      '130' : 'giardino',
                      '140' : 'recinzione'}

In [ ]:
#Tempo di default delle attività
#chiave: IdAttivita
#valore: tempo
tempo_attivita = {'inizio': 0,
                '010': 3, 
                '020': 3, 
                '030': 5,
                '040': 3, 
                '050': 5,
                '060': 3,
                '070': 2, 
                '080': 1, 
                '090': 4, 
                '100': 2, 
                '110': 2,
                '120': 2,
                '130': 4,
                '140': 4,
                'fine': 0}

In [ ]:
#Associazioni degli operai alle attività che possono svolgere 
#chiave: IdAttivita
#valore: lista di identificativi di operai
attivita_operai = {
    '010': ['0001', '0002','0029'],
    '020': ['0003', '0004','0030'],
    '030': ['0005', '0006','0031'],
    '040': ['0007', '0008','0032'],
    '050': ['0009', '0010','0033'],
    '060': ['0011', '0012','0034'],
    '070': ['0013', '0014','0035'],
    '080': ['0015', '0016','0036'],
    '090': ['0017', '0018','0037'],
    '100': ['0019', '0020','0038'],
    '110': ['0021', '0022','0039'],
    '120': ['0023', '0024','0040'],
    '130': ['0025', '0026','0041'],
    '140': ['0027', '0028','0042'],
}

In [ ]:
listaCase = []
N_case = int (input("Inserisci un numero di case: "))

for x in range(N_case):
  listaCase.append(tempo_attivita)

listaCase

In [ ]:
def modificaTempo():
    tempoCasa = {}
    for key in tempo_attivita:
        if (key == "inizio") or (key == "fine"):
            tempoCasa[key] = 0
        else:
            tempoCasa[key] = int (input ("Tempo per " + dizionario_attivita[key] + ":"))
    return tempoCasa

In [ ]:
risposteAffermative = ["si", "Si", "SI", "sI", "S", "s"]
risposteNegative = ["no", "No", "nO", "NO", "n", "N"]
risposta = ""
while (risposta not in risposteAffermative) and (risposta not in risposteNegative):
    risposta = input("Vuoi modificare i tempi delle attivita delle case?:  ")

if risposta in risposteAffermative:
    caseDaModificare = 0
    while 1:
        caseDaModificare = int (input ("Per quante case vuoi cambiare i tempi predefiniti: "))
        if caseDaModificare > 0 and caseDaModificare <= N_case:
            for i in range(caseDaModificare):
                print ("Inserisci i tempi per ogni attivita. Casa numero: ", i+1)
                listaCase[i] = modificaTempo()
            break;
        else:
            print ("Numero di case da modificare non consentito")

In [ ]:
listaCase

In [ ]:
#grafo che rappresenta l'ordine delle attività
grafo = { 'inizio': ['030', '120'],
          '030': ['040'],
          '120': ['040', '010', '140'],
          '040': ['070', '060', '050'],
          '010': ['020'],
          '140': ['130'],
          '070': ['100'],
          '060': ['090'],
          '050': ['090', '080'],
          '020': ['110'],
          '130': ['fine'],
          '100': ['110'],
          '090': ['110'],
          '080': ['110'],
          '110': ['fine'],
          'fine': []         
}

In [ ]:
#Ricavare una lista di tuple contentente l'attività e il giorno in cui termina (per ogni casa) 
from ricercaBfs import bfsMod, filtroTempoAttivita

fineAttivita = []

for i in range(N_case):
    fineAttivita.append ( bfsMod(grafo, ('inizio', '0'), listaCase[i]) )
    fineAttivita[i] = filtroTempoAttivita(fineAttivita[i], dizionario_attivita)

fineAttivita

In [ ]:
#Creazione delle variabili in una lista: variabili 
#Ogni variabile sarà una stringa del tipo:-  'GiornoInizioAttività'_'Casa'_'Attività'
variabili = []
for casa in range(N_case):
  for attivita in fineAttivita[casa]:
    tempoInizio = attivita[1] - listaCase[casa][attivita[0]] + 1

    while tempoInizio != attivita[1] + 1 :
      variabili.append ("G" + str(tempoInizio) + "_C" + str(casa) + "_" + attivita[0])
      tempoInizio += 1
      
variabili

In [ ]:
#Creazione di un dizionario contente le variabili e i loro domini
VarConDominio = {}
for variabile in variabili:
  VarConDominio[variabile] = attivita_operai[(variabile[len(variabile)-3:])]
VarConDominio

In [ ]:
from cspProblem import CSP, Constraint        
from operator import lt,ne,eq,gt

In [ ]:
#Definizione dei vincoli tra le variabili
vincoli = []
coppia = []
for var1 in VarConDominio:
    for var2 in VarConDominio:

        giornoVar1 = var1.index('_')
        giornoVar1 = var1[:giornoVar1]
        giornoVar2 = var2.index('_')
        giornoVar2 = var2[:giornoVar2]

    
        if (giornoVar1 == giornoVar2) and (var1!=var2):
            temp = [var1,var2]
            temp.reverse()
            if temp not in coppia:
                vincoli.append ( Constraint([var1, var2], ne) )
                temp.reverse()
                coppia.append(temp)


vincoli

In [ ]:
#Definizione dei vincoli tra le variabili
vincoli = []
for var1 in VarConDominio:
  for var2 in VarConDominio:

    giornoVar1 = var1.index('_')
    giornoVar1 = var1[:giornoVar1]
    giornoVar2 = var2.index('_')
    giornoVar2 = var2[:giornoVar2]

    if giornoVar1 == giornoVar2 and var1!=var2:
      vincoli.append ( Constraint([var1, var2], ne) )

vincoli

In [ ]:
csp = CSP (VarConDominio, vincoli )

In [ ]:
#Risoluzione CSP tramite ricerca
from searchGeneric import Searcher
from cspSearch import Search_from_CSP 


searcher1 = Searcher(Search_from_CSP(csp))
print(searcher1.search())  # prossima soluzione